In [1]:
%reload_ext autoreload
%autoreload 2
from fun import *
import polars as pl
import pandas as pd
import datetime as dt
import time 
import logging

logging = get_logger(log_file='test.log')

start_date = dt.date(2021,1,1)
end_date = dt.datetime.today()
end_date = dt.date(2021,12,31)

# 获取指定日期的日线数据
stock_data = read_day_data(start_date=start_date,end_date=end_date,file_path='ts_stock_all_data',fields=['code','trading_date','name','open','high','low','close','pct','pre_close','limit_up','limit_down','volume','amount','industry','float_mv'])
stock_data = stock_data.drop_nulls(subset=['open','close','pre_close','limit_up','limit_down'])
df = stock_data.to_pandas()

df.dtypes


code                    object
trading_date    datetime64[ms]
name                    object
open                   float64
high                   float64
low                    float64
close                  float64
pct                    float64
pre_close              float64
limit_up               float64
limit_down             float64
volume                 float64
amount                 float64
industry                object
float_mv               float64
dtype: object

In [2]:
# 读取对应的策略交割单数据
signals_df = pd.read_csv('信号文件/断板低开-5--2.5 20251230_233039(sma7).csv', encoding='utf-8-sig')
signals_df['trading_date'] = pd.to_datetime(signals_df['trading_date'])
# 只保留要写回的列
cols_to_keep = ["code", "trading_date", "buy_time", "sell_time", "buy_price", "sell_price", "sell_reason","signal"]
signals_df = signals_df[cols_to_keep]
# 左连接回全量 K 线
df = df.merge(signals_df, how='left', left_on=['code', 'trading_date'], right_on=['code', 'trading_date'])
df.set_index('trading_date',inplace=True)

In [3]:
# 假设add_sma已在pd_fun中定义，若未定义需补充：
"""
交互式K线生成 + HTML 导出
新增：在同一页面生成信号表（可排序、搜索）。
"""
import pandas as pd
from pd_fun import add_sma

def plot_interactive_kline(df, title='K线图', add_line_list=['sma_5','sma_10','sma_20']):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    df_plot = df.copy()
    if 'name' in df_plot.columns and 'code' in df_plot.columns:
        title = f"{df_plot['name'].iloc[0]}({df_plot['code'].iloc[0]})"
    for line in add_line_list:
        if line not in df_plot.columns:
            period = int(line.split('_')[1])
            df_plot = add_sma(df_plot, period)
    df_plot['color'] = df_plot.apply(lambda x: 'red' if x['close'] >= x['open'] else 'green', axis=1)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
    fig.add_trace(
        go.Candlestick(
            x=df_plot.index,
            open=df_plot['open'],
            high=df_plot['high'],
            low=df_plot['low'],
            close=df_plot['close'],
            increasing_line_color='red',
            decreasing_line_color='green',
            name='K线',
            hovertext=df_plot.apply(
                lambda x: (
                    f"时间: {x.name}<br>"
                    f"开盘: {x['open']:.2f}<br>"
                    f"最高: {x['high']:.2f}<br>"
                    f"最低: {x['low']:.2f}<br>"
                    f"收盘: {x['close']:.2f}<br>"
                    f"涨跌幅: {x['pct']:.2f}%"
                ), axis=1
            ),
            hoverinfo='text'
        ),
        row=1, col=1
    )
    for line in add_line_list:
        fig.add_trace(
            go.Scatter(
                x=df_plot.index,
                y=df_plot[line],
                mode='lines',
                name=line.upper(),
                line=dict(width=1.5),
                hovertext=df_plot.apply(lambda x, l=line: f"{l.upper()}: {x[l]:.2f}", axis=1),
                hoverinfo='text'
            ),
            row=1, col=1
        )
    if 'signal' in df_plot.columns:
        buy_df = df_plot[df_plot['signal'] == 1]
        sell_df = df_plot[df_plot['signal'] == -1]
        fig.add_trace(
            go.Scatter(
                x=buy_df.index,
                y=buy_df['low'],
                mode='markers',
                marker=dict(symbol='arrow-bar-up', color='dodgerblue', size=14, line=dict(width=1, color='black')),
                name='买入',
                hovertext=buy_df.apply(
                    lambda x: (
                        f"买入时间: {x['buy_time']}<br>"
                        f"买入价: {x['buy_price']:.2f}<br>"
                        f"卖出时间: {x['sell_time']}<br>"
                        f"卖出价: {x['sell_price']:.2f}<br>"
                        f"卖出原因: {x['sell_reason']}"
                    ), axis=1
                ),
                hoverinfo='text'
            ),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(
                x=sell_df.index,
                y=sell_df['high'],
                mode='markers',
                marker=dict(symbol='arrow-bar-down', color='orange', size=14, line=dict(width=1, color='black')),
                name='卖出',
                hovertext=sell_df.apply(
                    lambda x: (
                        f"买入时间: {x['buy_time']}<br>"
                        f"买入价: {x['buy_price']:.2f}<br>"
                        f"卖出时间: {x['sell_time']}<br>"
                        f"卖出价: {x['sell_price']:.2f}<br>"
                        f"卖出原因: {x['sell_reason']}"
                    ), axis=1
                ),
                hoverinfo='text'
            ),
            row=1, col=1
        )
    fig.add_trace(
        go.Bar(
            x=df_plot.index,
            y=df_plot['volume'],
            marker_color=df_plot['color'],
            name='成交量',
            hovertext=df_plot.apply(lambda x: f"时间: {x.name}<br>成交量: {x['volume']}", axis=1),
            hoverinfo='text'
        ),
        row=2, col=1
    )
    fig.update_layout(
        title=dict(text=title, x=0.05, y=0.95, xanchor='left', yanchor='top', font=dict(size=16)),
        legend=dict(x=0.75, y=1.05, xanchor='center', yanchor='top', bgcolor='rgba(255,255,255,0.8)', orientation='h', font=dict(size=10)),
        xaxis=dict(type='category', rangeslider=dict(visible=False), showgrid=True, gridcolor='lightgray', tickangle=45, tickmode='linear', dtick=15, tickfont=dict(size=8)),
        yaxis=dict(showgrid=True, gridcolor='lightgray'),
        hovermode='x unified',
        hoverlabel=dict(bgcolor='white', font=dict(size=12)),
        height=800,
        width=1200
    )
    fig.update_xaxes(type='category', title_text='时间', row=2, col=1, tickangle=45, tickmode='linear', dtick=15, tickfont=dict(size=8))
    fig.update_yaxes(title_text='价格', row=1, col=1)
    fig.update_yaxes(title_text='成交量', row=2, col=1)
    return fig

import plotly.io as pio

# -------- 生成所有股票的图和目录 --------
html_parts = []
toc_items = []
code_list = df[df['signal'] == 1]['code'].unique()

for code in code_list:
    df_stock = df[df['code'] == code]
    fig = plot_interactive_kline(
        df_stock,
        title=f"{df_stock['name'].iloc[0]}({code}) 日线图",
        add_line_list=['sma_5','sma_10','sma_20']
    )
    anchor = f"code-{code}"
    toc_items.append(f'<li><a href="#{anchor}">{df_stock["name"].iloc[0]} ({code})</a></li>')
    html_parts.append(f'<h2 id="{anchor}">{df_stock["name"].iloc[0]} ({code})</h2>')
    html_parts.append(pio.to_html(fig, include_plotlyjs='cdn', full_html=False))

toc_html = "<nav class='toc'><h2>目录</h2><ol>" + "".join(toc_items) + "</ol></nav>"

# -------- 生成信号表（DataTables，可排序/搜索/分页） --------
signals_df_for_table = signals_df.copy()
if 'trading_date' in signals_df_for_table.columns:
    signals_df_for_table['trading_date'] = pd.to_datetime(signals_df_for_table['trading_date']).dt.strftime('%Y-%m-%d')
for col in ['buy_time', 'sell_time']:
    if col in signals_df_for_table.columns:
        signals_df_for_table[col] = signals_df_for_table[col].astype(str)

table_html = signals_df_for_table.to_html(
    index=False,
    classes='sig-table',
    table_id='signals-table',
    border=0,
    justify='center'
)

page_size = 10  # 每页行数

full_html = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset='utf-8'/>
  <title>策略信号与图表</title>
  <style>
    body{{margin:0;font-family:Arial, sans-serif;}}
    .container{{display:flex;min-height:100vh;}}
    .toc{{width:220px;padding:16px;background:#f7f7f7;border-right:1px solid #e0e0e0;position:sticky;top:0;height:100vh;overflow:auto;}}
    .toc ol{{padding-left:18px;line-height:1.6;}}
    .main{{flex:1;padding:16px 24px;}}
    .table-wrap{{width:100%;overflow-x:auto;margin-bottom:24px;}}
    table{{border-collapse:collapse;width:100%;}}
    th, td{{border:1px solid #e0e0e0;padding:6px 8px;font-size:13px;white-space:nowrap;}}
    th{{background:#fafafa;cursor:pointer;}}
    tr:nth-child(even){{background:#fafafa;}}
    .controls{{margin:8px 0;display:flex;gap:12px;align-items:center;flex-wrap:wrap;}}
    .pager button{{margin:0 2px;}}
  </style>
</head>
<body>
<div class="container">
  {toc_html}
  <div class="main">
    <h1>信号表</h1>
    <div class="controls">
      <label>搜索: <input id="search-box" type="text" placeholder="输入关键词"/></label>
      <div class="pager">
        <button id="prev-btn">上一页</button>
        <span id="page-info"></span>
        <button id="next-btn">下一页</button>
      </div>
    </div>
    <div class="table-wrap">{table_html}</div>
    {''.join(html_parts)}
  </div>
</div>

<script>
(function(){{
  const table = document.getElementById('signals-table');
  const tbody = table.querySelector('tbody');
  let rows = Array.from(tbody.querySelectorAll('tr'));
  let filtered = rows.slice();
  let pageSize = {page_size};
  let page = 1;
  let sortCol = null, sortAsc = true;

  function cellValue(tr, idx){{
    const txt = tr.children[idx].innerText.trim();
    // 尝试数字
    const num = parseFloat(txt.replace(/,/g,''));
    if(!isNaN(num)) return num;
    // 尝试日期
    const t = Date.parse(txt);
    if(!isNaN(t)) return t;
    return txt.toLowerCase();
  }}

  function render(){{
    const start = (page-1)*pageSize;
    const pageRows = filtered.slice(start, start+pageSize);
    tbody.innerHTML = '';
    pageRows.forEach(r => tbody.appendChild(r));
    document.getElementById('page-info').innerText =
      `第 ${{page}} / ${{Math.max(1, Math.ceil(filtered.length/pageSize))}} 页，共 ${{filtered.length}} 条`;
  }}

  function sortBy(idx){{
    if(sortCol === idx) sortAsc = !sortAsc;
    else {{ sortCol = idx; sortAsc = true; }}
    filtered.sort((a,b) => {{
      const va = cellValue(a, idx), vb = cellValue(b, idx);
      if(va < vb) return sortAsc ? -1 : 1;
      if(va > vb) return sortAsc ? 1 : -1;
      return 0;
    }});
    page = 1;
    render();
  }}

  function filter(){{
    const kw = document.getElementById('search-box').value.toLowerCase();
    if(!kw) filtered = rows.slice();
    else {{
      filtered = rows.filter(tr => tr.innerText.toLowerCase().includes(kw));
    }}
    page = 1;
    render();
  }}

  table.querySelectorAll('th').forEach((th, idx) => {{
    th.addEventListener('click', () => sortBy(idx));
  }});
  document.getElementById('search-box').addEventListener('input', filter);
  document.getElementById('prev-btn').addEventListener('click', () => {{
    if(page > 1) {{ page--; render(); }}
  }});
  document.getElementById('next-btn').addEventListener('click', () => {{
    if(page < Math.ceil(filtered.length/pageSize)) {{ page++; render(); }}
  }});

  render();
}})();
</script>
</body>
</html>
"""


with open("kline_all_stock_2021.html", "w", encoding="utf-8") as f:
    f.write(full_html)
print("已生成 kline_all_stock.html，双击或浏览器打开查看。")

已生成 kline_all_stock.html，双击或浏览器打开查看。
